In [ ]:
# initialization
import tensorflow as tf
from tensorflow import keras

# test for GPU use in tensorflow
physical_gpus = tf.config.list_physical_devices('GPU')
print(physical_gpus)

# define variables and constants
DIMX = 256
IMAGE_SIZE = (DIMX, DIMX)
BATCH_SIZE = 32
EPOCHS = 30

BASE_DIR = './'
IMAGE_DIR = BASE_DIR + 'images/'
MODEL_DIR = BASE_DIR + 'models/'
RESULTS_DIR = BASE_DIR + 'results/'
DATASET_DIR = '../'
USE_CPU = False

if USE_CPU:
    tf.config.set_visible_devices([], 'GPU')

print("Tensorflow version", tf.__version__)
print("Keras version", keras.__version__)

In [ ]:
# plot loss and accuracy graphics

import matplotlib.pyplot as plt
    
def draw_loss_acc_graphics(outfile, model_name, history):
    acc = history.history['accuracy']
    loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc)+1)

    plt.figure()
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'c', label='Validation acc')
    plt.title("Training and Validation Accuracy")
    plt.legend()
    plt.savefig(outfile + '_accuracy_graph.pdf')  
    plt.close()


    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'c', label='Validation loss')
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.savefig(outfile + '_loss_graph.pdf')
    plt.close()




In [ ]:
# evaluate model with test set

from sklearn.metrics import ConfusionMatrixDisplay, classification_report
import sklearn.metrics as metric
import numpy as np

def process_tests(model, test_ds):
    y_true = np.array([], dtype=np.int32)
    y_pred = np.array([], dtype=np.int32)
    images = []
    
    for image_batch, labels in test_ds:
        y_true = np.concatenate([y_true, np.argmax(labels.numpy(), axis=-1)])
        y_pred = np.concatenate(
            [y_pred, np.argmax(model.predict(image_batch,verbose=0), axis=-1)]
        )
        for img in image_batch:
            images.append(img)

    size_test = sum([len(l) for _,l in test_ds])
    images = np.reshape(images, (size_test,DIMX,DIMX,3))
   
    return y_true, y_pred, images
    
def draw_confusion_matrix(outfile, model, model_name, test_ds, class_names):
    
    # plot confusion matrix
    y_true, y_pred, images = process_tests(model, test_ds)
    plt.figure(figsize=(5, 5))
    cmap = plt.cm.Blues #'viridis'

    # save confusion matrix with classification percentage
    cm = ConfusionMatrixDisplay.from_predictions(
        y_true, y_pred, cmap=cmap, labels=range(len(class_names)), 
        display_labels=class_names, normalize="pred" 
    )
    plt.tight_layout()
    ax = plt.gca()
    plt.setp(ax.yaxis.get_majorticklabels(), rotation=90, ha="center", rotation_mode="anchor") 
    plt.savefig(outfile + '_cm.pdf')

    # save confusion matrix with number of samples
    cm = ConfusionMatrixDisplay.from_predictions(
        y_true, y_pred, cmap=cmap, labels=range(len(class_names)), 
        display_labels=class_names
    )
    plt.savefig(outfile + '_cm2.pdf')
    plt.close()

    # calculate accuracy, precision, recall and F1-score
    accuracy = metric.accuracy_score(y_true, y_pred)
    precision = metric.precision_score(y_true, y_pred, average="macro")
    recall = metric.recall_score(y_true, y_pred, average="macro")
    f1 = metric.f1_score(y_true, y_pred, average="macro")
    print('Accuracy | Precision | Recall | F1 score')
    print(accuracy, precision, recall, f1)
    print(metric.precision_recall_fscore_support(y_true, y_pred, average="macro"))
    
    report = classification_report(
        y_true, y_pred, labels=range(len(class_names)), 
        target_names=class_names
    )
    print(report)
    
    return precision, recall, f1, report


In [ ]:

import numpy as np

def save_misclassified_images(outfile, model, model_name, test_ds, class_names, n_images=3):
    
    # save incorrectly classified test images 
    y_true, y_pred, images = process_tests(model, test_ds)

    a = np.array(y_true) == np.array(y_pred)
    plt.figure()
    ax=plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    j=0
    for i in range(len(a)):
        if not a[i] and j<n_images:
            j=j+1
            plt.imshow(np.array(images[i]).astype("uint8"))
            plt.title(class_names[y_true[i]] + " - " + class_names[y_pred[i]])
            plt.tight_layout()
            plt.savefig(outfile + '_misclassified_' + str(j) + '.pdf')
    plt.close()

def save_results(
    filename, model, model_name, train_ds, val_ds, test_ds, 
    class_names, history
):
    # loss-accuracy graphics
    draw_loss_acc_graphics(IMAGE_DIR+filename, model_name, history)

    # confusion matrix
    precision, recall, f1, report = draw_confusion_matrix(
        IMAGE_DIR+filename, model, model_name, test_ds, class_names
    )   

    # misclassified images
    save_misclassified_images(
        IMAGE_DIR+'misclassified/'+filename, model, 
        model_name, test_ds, class_names
    )

    # save results to file
    train_loss, train_acc = model.evaluate(train_ds)
    val_loss, val_acc     = model.evaluate(val_ds)
    test_loss, test_acc   = model.evaluate(test_ds)

    with open(RESULTS_DIR+filename+'.txt', 'w') as f:
        f.write("Training loss/accuracy: " + str(train_loss) + ', '\
                + str(train_acc) + '\n')
        f.write("Validate loss/accuracy: " + str(val_loss) + ', '\
                + str(val_acc) + '\n')
        f.write("Testing  loss/accuracy: " + str(test_loss) + ', '\
                + str(test_acc) + '\n')
        f.write("Precision: " + str(precision) + '\n')
        f.write("Recall: " + str(recall) + '\n')
        f.write("F1: " + str(f1) + '\n')
        f.write(report + '\n')


In [ ]:
import os
import tensorflow as tf
import keras.backend as K
      
def run_experiments(
    dataset_name, model_name, train_ds, val_ds, test_ds, 
    class_names, epochs=EPOCHS, data_augmentation=False, 
    transfer_learning=False
):

    model = create_model(
        model_name, len(class_names), DIMX, BATCH_SIZE, 
        data_augmentation, transfer_learning
    )

    model.summary()

    history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=val_ds,
        verbose=1
    )
    
    # save model to disk
    filename = model_name + '_' + dataset_name 
    if data_augmentation:
        filename = filename + '_DA'
    if transfer_learning:
        filename = filename + '_TL'

    # save results and graphics to disk 
    save_results(
        filename,model,model_name,train_ds,val_ds,
        test_ds, class_names, history
    )
    
    if transfer_learning and not data_augmentation:
        # do fine-tuning after transfer learning
        print("Fine-tuning the model")
        # model.layers[1].trainable=True #full fine-tuning
        for layer in model.layers[1].layers[-15:]:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        model.compile(
            optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
        )
        model.summary()

        fine_tuned_history = model.fit(
            train_ds.concatenate(val_ds), epochs=15, batch_size=BATCH_SIZE,
            validation_data=test_ds, 
            verbose=1
        )

        # save fine-tuned results 
        save_results(
            filename + '_fine-tuned', model, model_name,
            train_ds, val_ds, test_ds, class_names, fine_tuned_history,
        )
    
    
    
def experiments_with_dataset(dataset, model_list, train_ds, val_ds, test_ds, class_names):
    
    print("Number of classes", len(class_names))

    for model_name in model_list:
        print('# ' + model_name + ' model')
         
        # training from scratch
        print('  ## training from scratch')
        run_experiments(
            dataset, model_name, train_ds, val_ds, 
            test_ds, class_names, EPOCHS
        )
        
        # using data augmentation
        print('  ## training with data augmentation')
        run_experiments(
            dataset, model_name, train_ds, val_ds, 
            test_ds, class_names, min(100,round(1.5*EPOCHS)), True
        )
        
        if model_name != 'custom':
            
            # using transfer learning
            print('  ## training with transfer learning')
            run_experiments(
                dataset, model_name, train_ds, val_ds, 
                test_ds, class_names, EPOCHS, False, True
            )
            
            # using transfer learning and data augmentation
            print('  ## training with transfer learning and data augmentation')
            run_experiments(
                dataset, model_name, train_ds, val_ds, 
                test_ds, class_names, min(100,round(1.5*EPOCHS)), True, True
            )

            

In [ ]:
from load_datasets import load
from create_model import create_model

from tensorflow.keras import models
from tensorflow.keras import layers

datasets = ['figshare', 'kaggle']

model_list = ['custom', 'vgg16', 'vgg19', 
              'resnet50v2', 'resnet101v2','resnet152v2', 
              'xception', 'densenet', 'mobilenet', 
              'efficientnetb0v2','efficientnetb3v2',
              'convnexttiny'
             ]
m1 = 0
m2 = None

# experiments with Figshare
print('\nProcessing Figshare Dataset:')
print('--------------------------------')
train_ds1, val_ds1, test_ds1, class_names1 = load(
    datasets[0], DATASET_DIR, IMAGE_SIZE, BATCH_SIZE, True
)

experiments_with_dataset(
    datasets[0], model_list[m1:m2], train_ds1, 
    val_ds1, test_ds1, class_names1
)

# experiments with Kaggle
print('\nProcessing Kaggle Dataset:')
print('--------------------------------')
train_ds2, val_ds2, test_ds2, class_names2 = load(
    datasets[1], DATASET_DIR, IMAGE_SIZE, BATCH_SIZE, True
)

experiments_with_dataset(
    datasets[1], model_list[m1:m2], train_ds2, 
    val_ds2, test_ds2, class_names2
)
